In [44]:
import pandas as pd
import numpy as np

season_data = pd.read_csv("pbp_2017.csv", low_memory=False)


In [45]:
#Relevant Columns

cols = ['Date',
'GameID',
'Drive',
'qtr',
'down',
'TimeSecs',
'PlayTimeDiff',
'SideofField',
'yrdline100',
'ydstogo',
'ydsnet',
'GoalToGo',
'FirstDown',
'posteam',
'DefensiveTeam',
'PlayAttempted',
'Yards.Gained',
'sp',
'Touchdown',
'ExPointResult',
'TwoPointConv',
'DefTwoPoint',
'Safety',
'PlayType',
'Passer',
'Passer_ID',
'PassAttempt',
'PassOutcome',
'PassLength',
'AirYards',
'YardsAfterCatch',
'QBHit',
'PassLocation',
'InterceptionThrown',
'Rusher',
'Rusher_ID',
'RushAttempt',
'RunLocation',
'RunGap',
'Receiver',
'Receiver_ID',
'Reception',
'Fumble',
'RecFumbTeam',
'Sack',
'PosTeamScore',
'DefTeamScore',
'ScoreDiff',
'AbsScoreDiff',
'HomeTeam',
'AwayTeam',
'Season']

season_data = season_data[cols]

In [46]:
#TOTAL Season Passing Points

def passing_points(passer):
    #Establishing Dataframes
    passer_df = season_data[season_data['Passer']==passer]
    passing_df = passer_df[passer_df['PassOutcome']=='Complete']
    
    #Total Passing Yards
    pd.to_numeric(passing_df['AirYards'])
    pd.to_numeric(passing_df['YardsAfterCatch'])
    
    total_pass_yards = passing_df['AirYards'].sum() + passing_df['YardsAfterCatch'].sum()
    total_pass_points = .04 * total_pass_yards
    
    #Interceptions
    pd.to_numeric(passer_df['InterceptionThrown'])
    total_interceptions = passer_df['InterceptionThrown'].sum()
    total_interception_points = (-1) * total_interceptions
    
    #Passing Touchdowns
    pd.to_numeric(passing_df['Touchdown'])
    total_pass_td = passing_df['Touchdown'].sum()
    total_pass_td_points = 4 * total_pass_td

    #300+ Yard Games
    game_list = passing_df['GameID'].unique().tolist()
    three_hund_plus = 0
    
    for game in game_list:
        game_df = passing_df[passing_df['GameID'] == game]
        game_yards = game_df['AirYards'].sum() + game_df['YardsAfterCatch'].sum()
        if game_yards >= 300:
            three_hund_plus += 1
    
    total_three_hund_plus = 3 * three_hund_plus 
    
    #Total DK Passing Points
    total_dk_passing_points = total_pass_points + total_interception_points + total_pass_td_points + total_three_hund_plus
    return total_dk_passing_points


In [47]:
#TOTAL Season Rushing Points

def rushing_points(rusher):
    #Establishing Dataframe
    rusher_df = season_data[season_data['Rusher']==rusher]
    
    #Rushing Touchdowns
    pd.to_numeric(rusher_df['Touchdown'])
    total_rushing_touchdowns = rusher_df['Touchdown'].sum()
    total_rushing_td_points = 6 * total_rushing_touchdowns
    
    #Rushing Yards
    pd.to_numeric(rusher_df['Yards.Gained'])
    total_rushing_yards = rusher_df['Yards.Gained'].sum()
    total_rushing_yd_points = 0.1 * total_rushing_yards
    
    #100+ Yard Games
    game_list = rusher_df['GameID'].unique().tolist()
    one_hund_plus = 0
    
    for game in game_list:
        game_df = rusher_df[rusher_df['GameID'] == game]
        game_yards = game_df['Yards.Gained'].sum()
        if game_yards >= 100:
            one_hund_plus += 1
    
    total_one_hund_plus = 3 * one_hund_plus 
    
    #Fumbles
    pd.to_numeric(rusher_df['Fumble'])
    total_fumbles = rusher_df['Fumble'].sum()
    total_fumbles_points = (-1) * total_fumbles
    
    
    #Total DK Rushing Points
    total_dk_rushing_points = total_rushing_td_points + total_rushing_yd_points + total_one_hund_plus + total_fumbles_points
    
    return(total_dk_rushing_points)


In [48]:
#TOTAL Season Receiving Points

def receiver_points(receiver):
    #Establishing Dataframe
    receiver_df = season_data[season_data['Receiver']==receiver]
    reception_df = receiver_df[receiver_df['PassOutcome']=='Complete']
    
    #Receiving Touchdowns
    pd.to_numeric(receiver_df['Touchdown'])
    total_receiving_touchdowns = receiver_df['Touchdown'].sum()
    total_receiving_td_points = 6 * total_receiving_touchdowns
    
    #Receiving Yards
    total_receiving_yards = reception_df['AirYards'].sum() + reception_df['YardsAfterCatch'].sum()
    total_receiving_yd_points = 0.1 * total_receiving_yards
    
    #100+ Yard Games
    game_list = reception_df['GameID'].unique().tolist()
    one_hund_plus = 0
    
    for game in game_list:
        game_df = reception_df[reception_df['GameID'] == game]
        game_yards = game_df['AirYards'].sum() + game_df['YardsAfterCatch'].sum()
        if game_yards >= 100:
            one_hund_plus += 1
    
    total_one_hund_plus = 3 * one_hund_plus 
    
    #PPR
    receptions = len(reception_df)
    
    #Fumbles
    pd.to_numeric(reception_df['Fumble'])
    total_fumbles = reception_df['Fumble'].sum()
    total_fumbles_points = (-1) * total_fumbles
    
    
    #Total DK Receiver Points
    total_dk_receiving_points = total_receiving_td_points + total_receiving_yd_points + total_one_hund_plus + total_fumbles_points + receptions  
    
    return(total_dk_receiving_points)


In [49]:
#Just Defensive Score Points

def score_against(team):
    team_df = season_data[(season_data["HomeTeam"]==team) | (season_data['AwayTeam']==team)]
    game_list = team_df['GameID'].unique().tolist()
    
    score_points = 0
    
    for game in game_list:
        game_df = team_df[team_df['GameID']==game]
        game_df = game_df[['posteam', 'DefensiveTeam', 'PosTeamScore', 'DefTeamScore']]
        game_df = game_df.dropna()
        
        game_length = len(game_df) -1 
        last_play = game_df.iloc[game_length]
        
        if last_play['posteam'] == team:
            points_given_up = last_play['DefTeamScore']
        else:
            points_given_up = last_play['PosTeamScore']
        
        if points_given_up == 0:
            score_points += 10
        elif points_given_up >= 1 and points_given_up <= 6:
            score_points += 7
        elif points_given_up >= 7 and points_given_up <= 13:
            score_points += 4
        elif points_given_up >= 14 and points_given_up <= 20:
            score_points += 1
        elif points_given_up >= 21 and points_given_up <= 27:
            score_points += 0
        elif points_given_up >= 28 and points_given_up <= 34:
            score_points -= 1
        else:
            score_points -= 4
    
    return score_points           

In [50]:
#TOTAL Season Defense Points
#TEAM ABBREVIATIONS
defense_list = ['KC','NE','NYJ','BUF','PHI','WAS','TEN','OAK','HOU','JAX','DET','ARI','PIT','CLE','CHI','ATL','BAL','CIN','IND','LA','SF','CAR','GB','SEA','NYG','DAL','MIN','NO','LAC','DEN','TB','MIA']

def team_defense(team):
    defense_df = season_data[season_data['DefensiveTeam']==team]
    touchdown_df = defense_df[defense_df['Touchdown'] == 1]
    
    #SACKS
    total_sacks = defense_df['Sack'].sum()
    
    #INTERCEPTIONS
    total_interceptions = defense_df['InterceptionThrown'].sum()
    total_interception_points = 2 * total_interceptions
    
    #TDs OFF INTERCEPTIONS
    int_return_td = touchdown_df['InterceptionThrown'].sum()
    int_return_td_points = 6 * int_return_td

    #FUMBLE RECOVERIES
    total_recoveries = 0
    fumble_series = defense_df['RecFumbTeam'].dropna()

    for recovery in fumble_series:
        if recovery == team:
            total_recoveries += 1
    fumble_recovery_points = 2 * total_recoveries 

    #SAFETY
    total_safeties = defense_df['Safety'].sum()
    
    safety_points = 2 * total_safeties
    
    #Defensive Score Points
    total_defensive_score_points = score_against(team)
    
    #TOTAL DEFENSIVE POINTS
    total_dk_defense_points = total_sacks + total_interception_points + int_return_td_points + fumble_recovery_points + safety_points + total_defensive_score_points
    return(total_dk_defense_points)

DST_ranks = {}
for team in defense_list:
    points = team_defense(team)
    DST_ranks[team] = points

#Produces dataframe of just Defenses and their overall season points
DST_ranks_df = pd.DataFrame.from_dict(DST_ranks, orient='index', columns=['Total Points'])
pd.to_numeric(DST_ranks_df['Total Points'])

#Prettying Up The Data Frame Holy Shit This Was Annoying
DST_ranks_df = DST_ranks_df['Total Points'].sort_values(ascending=False)
DST =  pd.DataFrame(data=DST_ranks_df)
DST['Team'] = DST.index
DST.reset_index(inplace=True, level=None)
DST = DST.drop(['index'], axis=1)
DST = DST[['Team', 'Total Points']]


In [51]:
def total_offensive_points(player):
    passing_pts = passing_points(player)
    rushing_pts = rushing_points(player)
    receiving_pts = receiver_points(player)
    
    total_offensive_points = passing_pts + rushing_pts + receiving_pts
    
    return(total_offensive_points)

In [52]:
#Building DF for top 30ish quarterbacks
quarterbacks = season_data['Passer'].dropna().unique().tolist()
#Players that are either not full-time quarterbacks or are obvious errors (T.Hill and T.Taylor)
non_quarterbacks = ['C.Kupp','T.Cohen','M.Lee', 'J.Landry', 'L.Fitzgerald', 'S.Vereen', 'T.Kelce', 'G.Tate', 'T.Taylor', 'T.Hill', 'M.Sanu', 'R.Cobb', 'E.Decker', 'D.Stanton', 'M.Moore', 'M.Glennon', 'T.Yates', 'K.Hogan', 'N.Foles', 'S.Bradford', 'B.Petty', 'D.Fales', 'P.Lynch', 'E.Manuel', 'P.Mahomes', 'T.McEvoy', 'R.Mallett', 'N.Peterman', 'G.Smith', 'L.Jones', 'T.Bray', 'M.Cassel', 'S.Mannion', 'N.Sudfeld', 'J.Webb', 'C.Kessler', 'S.Tolzien', 'M.Gray', 'A.McCarron', 'C.Jones', 'C.Rush', 'R.Golden', 'S.Koch', 'J.Hekker', 'L.Edwards', 'B.Nortman', 'D.Anderson', 'T.Heinicke', 'J.Callahan', 'K.Clemens', 'R.Quigley', 'C.Henne', 'M.Palardy', 'M.Hull', 'J.Rudock', 'J.Ryan', 'T.Bridgewater', 'M.Haack']

for non_qb in non_quarterbacks:
    try:
        quarterbacks.remove(non_qb)
    except:
        pass

qb_ranks = {}
for player in quarterbacks:
    points = total_offensive_points(player)
    qb_ranks[player] = points

#Produces dataframe of just quarterbacks and their overall season points
qb_ranks_df = pd.DataFrame.from_dict(qb_ranks, orient='index', columns=['Total Points'])
pd.to_numeric(qb_ranks_df['Total Points'])

#Prettying up the QB Dataframe
qb_ranks_df = qb_ranks_df['Total Points'].sort_values(ascending=False)
quarterbacks = pd.DataFrame(data=qb_ranks_df)
quarterbacks['Player'] = quarterbacks.index
quarterbacks.reset_index(inplace=True, level=None)
quarterbacks = quarterbacks.drop(['index'], axis=1)
quarterbacks = quarterbacks[['Player', 'Total Points']]

In [68]:
#Season Broken into weeks
week_one = ['2017-09-07','2017-09-10','2017-09-11']
week_two = ['2017-09-14', '2017-09-17','2017-09-18' ]
week_three = ['2017-09-21', '2017-09-24', '2017-09-25']
week_four = ['2017-09-28', '2017-10-01', '2017-10-02']
week_five = ['2017-10-05', '2017-10-08', '2017-10-09']
week_six = ['2017-10-12', '2017-10-15', '2017-10-16']
week_seven = ['2017-10-19', '2017-10-22', '2017-10-23']
week_eight = ['2017-10-26', '2017-10-29', '2017-10-30']
week_nine = ['2017-11-02', '2017-11-05', '2017-11-06' ]
week_ten = ['2017-11-09', '2017-11-12', '2017-11-13']
week_eleven = ['2017-11-16', '2017-11-19', '2017-11-20']
week_twelve = ['2017-11-23', '2017-11-26', '2017-11-27']
week_thirteen = ['2017-11-30', '2017-12-03', '2017-12-04']
week_fourteen = ['2017-12-07', '2017-12-10', '2017-12-11']
week_fifteen = ['2017-12-14', '2017-12-16', '2017-12-17']
week_sixteen = ['2017-12-18', '2017-12-23', '2017-12-24', '2017-12-25']
week_seventeen = ['2017-12-31']

all_the_weeks_past_one = [week_two, week_three, week_four, week_five, week_six, week_seven, week_eight, week_nine, week_ten, week_eleven, week_twelve, week_thirteen, week_fourteen, week_fifteen, week_sixteen, week_seventeen]

def weekly_offense_points(player, week):

    current_week_df = pd.DataFrame()
    for game_day in week:
        game_day_df = season_data[season_data['Date']==game_day]
        current_week_df = pd.concat([current_week_df, game_day_df], ignore_index=True)
    
    #Passing Time Again
    passer_df = current_week_df[current_week_df['Passer']==player]
    passing_df = passer_df[passer_df['PassOutcome']=='Complete']
    
    opponent = str(passing_df['DefensiveTeam'].unique())
    opponent = opponent.replace("'","")
    opponent = opponent.replace("[", "")
    opponent = opponent.replace("]","")
    
    #Home and Away Teams
    away_team = str(passer_df['AwayTeam'].unique())
    away_team = away_team.replace("'","")
    away_team = away_team.replace("[", "")
    away_team = away_team.replace("]","")
    home_team = str(passer_df['HomeTeam'].unique())
    home_team = home_team.replace("'","")
    home_team = home_team.replace("[", "")
    home_team = home_team.replace("]","")
    
    #Total Passing Yards
    pd.to_numeric(passing_df['AirYards'])
    pd.to_numeric(passing_df['YardsAfterCatch'])
    
    total_pass_yards = passing_df['AirYards'].sum() + passing_df['YardsAfterCatch'].sum()
    total_pass_points = .04 * total_pass_yards
    
    #Interceptions
    pd.to_numeric(passer_df['InterceptionThrown'])
    total_interceptions = passer_df['InterceptionThrown'].sum()
    total_interception_points = (-1) * total_interceptions
    
    #Passing Touchdowns
    pd.to_numeric(passing_df['Touchdown'])
    total_pass_td = passing_df['Touchdown'].sum()
    total_pass_td_points = 4 * total_pass_td

    #Rushing Stuff
    rusher_df = current_week_df[current_week_df['Rusher']==player]
    
    #Rushing Touchdowns
    pd.to_numeric(rusher_df['Touchdown'])
    total_rushing_touchdowns = rusher_df['Touchdown'].sum()
    total_rushing_td_points = 6 * total_rushing_touchdowns
    
    #Rushing Yards
    pd.to_numeric(rusher_df['Yards.Gained'])
    total_rushing_yards = rusher_df['Yards.Gained'].sum()
    total_rushing_yd_points = 0.1 * total_rushing_yards
    
    #Fumbles
    pd.to_numeric(rusher_df['Fumble'])
    total_fumbles = rusher_df['Fumble'].sum()
    total_fumbles_points = (-1) * total_fumbles
    
    #Catch Dat Ball
    receiver_df = current_week_df[current_week_df['Receiver']==player]
    reception_df = receiver_df[receiver_df['PassOutcome']=='Complete']
    
    #Receiving Touchdowns
    pd.to_numeric(receiver_df['Touchdown'])
    total_receiving_touchdowns = receiver_df['Touchdown'].sum()
    total_receiving_td_points = 6 * total_receiving_touchdowns
    
    #Receiving Yards
    total_receiving_yards = reception_df['AirYards'].sum() + reception_df['YardsAfterCatch'].sum()
    total_receiving_yd_points = 0.1 * total_receiving_yards
    
    #PPR
    receptions = len(reception_df)
    
    #Fumbles after reception
    pd.to_numeric(reception_df['Fumble'])
    total_fumbles = reception_df['Fumble'].sum()
    total_fumbles_points_rec = (-1) * total_fumbles

    #Sum that shit up
    total_week_points = total_pass_points + total_interception_points + total_pass_td_points + total_rushing_td_points + total_rushing_yd_points + total_fumbles_points + total_receiving_td_points + total_receiving_yd_points + total_fumbles_points_rec + receptions
    
    #Motherfucking conditionals
    if total_pass_yards >= 300:
        total_week_points += 3
    
    if total_rushing_yards >= 100:
        total_week_points+= 3
    
    if total_receiving_yards >= 100:
        total_week_points += 3
    
    
    weekly_offense_stats = [total_week_points, total_pass_yards, total_pass_td, total_interceptions, total_rushing_yards, total_rushing_touchdowns, opponent, home_team, away_team]
    
    return(weekly_offense_stats)

print(weekly_offense_points('M.Stafford', week_one))
    

[28.18, 292, 4, 1, 15, 0, 'ARI', 'DET', 'ARI']


In [59]:
#season averages columns

def qb_season_averages(player):
    passer_df = season_data[season_data['Passer']==player]
    passing_df = passer_df[passer_df['PassOutcome']=='Complete']
    qb_averages = []
    
    games_played = len(passing_df['Date'].unique())
    
    #Total Pass Attempts
    total_pass_attempts = len(passer_df)
    avg_pass_attempts = total_pass_attempts / float(games_played)
    
    #Total Completions
    total_completions = len(passing_df)
    avg_completions = total_completions / float(games_played)
    
    #Total Passing Yards
    pd.to_numeric(passing_df['AirYards'])
    pd.to_numeric(passing_df['YardsAfterCatch'])
    
    total_pass_yards = passing_df['AirYards'].sum() + passing_df['YardsAfterCatch'].sum()
    average_pass_yards = total_pass_yards / float(games_played)
    
    #Deep Ball Passes
    deep_ball_pass_df = passer_df[passer_df['PassLength']=='Deep']
    deep_targets = len(deep_ball_pass_df)
    avg_deep_targets = deep_targets / float(games_played)
    
    #Interceptions
    pd.to_numeric(passer_df['InterceptionThrown'])
    total_interceptions = passer_df['InterceptionThrown'].sum()
    average_interceptions = total_interceptions / float(games_played)
    
    #Passing Touchdowns
    pd.to_numeric(passing_df['Touchdown'])
    total_pass_td = passing_df['Touchdown'].sum()
    average_pass_tds = total_pass_td / float(games_played)

    #Rushing Stuff
    rusher_df = season_data[season_data['Rusher']==player]
    
    #Rushing Touchdowns
    pd.to_numeric(rusher_df['Touchdown'])
    total_rushing_touchdowns = rusher_df['Touchdown'].sum()
    average_rush_tds = total_rushing_touchdowns / float(games_played)
    
    #Rushing Yards
    pd.to_numeric(rusher_df['Yards.Gained'])
    total_rushing_yards = rusher_df['Yards.Gained'].sum()
    average_rush_yards = total_rushing_yards / float(games_played)
    
    #Red Zone Opportunities
    pd.to_numeric(passer_df['yrdline100'])
    pd.to_numeric(rusher_df['yrdline100'])
    red_zone_pass_df = passer_df[passer_df['yrdline100'] >= 80]
    red_zone_rush_df = rusher_df[rusher_df['yrdline100'] >= 80]
    red_zone_pass_opps = len(red_zone_pass_df)
    red_zone_rush_opps = len(red_zone_rush_df) 
    red_zone_opps = red_zone_pass_opps + red_zone_rush_opps
    #Avg Red Zone Opps
    avg_red_zone_opps = red_zone_opps / float(games_played)
    
    qb_averages = [total_pass_attempts, avg_pass_attempts, total_completions, avg_completions, average_pass_yards, average_pass_tds, average_interceptions, average_rush_yards, average_rush_tds,  red_zone_pass_opps, red_zone_rush_opps, avg_red_zone_opps]

    return(qb_averages)

quarterbacks_list = quarterbacks['Player'].tolist()

def average_columns(val):
    average_calc = []
    for qb in quarterbacks_list:
        average_calc.append(qb_season_averages(qb)[val])
        
    return(average_calc)

quarterbacks['Total Pass Attempts'] = average_columns(0)
quarterbacks['Average Pass Attempts'] = average_columns(1)
quarterbacks['Total Completions'] = average_columns(2)
quarterbacks['Average Completions'] = average_columns(3)
quarterbacks['Average Pass Yds'] = average_columns(4)
quarterbacks['Average Pass Tds'] = average_columns(5)
quarterbacks['Average Interceptions'] = average_columns(6)
quarterbacks['Average Rush Yds'] = average_columns(7)
quarterbacks['Average Rush Tds'] = average_columns(8)
quarterbacks['Red Zone Pass Opps'] = average_columns(9)
quarterbacks['Red Zone Rush Opps'] = average_columns(10)
quarterbacks['Avg Red Zone Opps'] = average_columns(11)



    

In [69]:
#quarterbacks weekly data

def column_creator(val):
    current_week_pts = []
    for qb in quarterbacks_list:
        current_week_pts.append(weekly_offense_points(qb,week_one)[val])

    return(current_week_pts)

quarterbacks['Week 1 pts'] = column_creator(0)
quarterbacks['Week 1 Pass Yds'] = column_creator(1)
quarterbacks['Week 1 Pass TDs'] = column_creator(2)
quarterbacks['Week 1 Interceptions'] = column_creator(3)
quarterbacks['Week 1 Rush Yds'] = column_creator(4)
quarterbacks['Week 1 Rush TDs'] = column_creator(5)
quarterbacks['Week 1 Opponent'] = column_creator(6)
quarterbacks['Week 1 Home Team'] = column_creator(7)
quarterbacks['Week 1 Away Team'] = column_creator(8)

for i in range(2,17):
    week = all_the_weeks_past_one[i-2]
    def column_creator_v2(val):
        current_week_pts = []
        for qb in quarterbacks_list:
            current_week_pts.append(weekly_offense_points(qb, week)[val])
        
        return(current_week_pts)
        
    quarterbacks['Week '+ str(i) + ' pts'] = column_creator_v2(0)
    quarterbacks['Week '+ str(i) + ' Pass Yds'] = column_creator_v2(1)
    quarterbacks['Week '+ str(i) + ' Pass TDs'] = column_creator_v2(2)
    quarterbacks['Week '+ str(i) + ' Inteceptions'] = column_creator_v2(3)
    quarterbacks['Week '+ str(i) + ' Rush Yds'] = column_creator_v2(4)
    quarterbacks['Week '+ str(i) + ' Rush TDs'] = column_creator_v2(5)
    quarterbacks['Week '+ str(i) + ' Opponent'] = column_creator_v2(6)
    quarterbacks['Week '+ str(i) + ' Home Team'] = column_creator_v2(7)
    quarterbacks['Week '+ str(i) + ' Away Team'] = column_creator_v2(8)
        



In [70]:
quarterbacks.to_csv('quarterbacks_2017_home_away3.csv', sep=',')